<a href="https://colab.research.google.com/github/Daivar/PP6_Adult_dataset/blob/main/Adult_dataset_fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from fastai.tabular import *

!rm -rf ./sample_data/ ./data/

path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')

In [2]:
df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,36,Private,297449,Bachelors,13.0,Divorced,Prof-specialty,Not-in-family,White,Male,14084,0,40,United-States,>=50k
32557,23,?,123983,Bachelors,13.0,Never-married,?,Own-child,Other,Male,0,0,40,United-States,<50k
32558,53,Private,157069,Assoc-acdm,12.0,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>=50k
32559,32,Local-gov,217296,HS-grad,9.0,Married-civ-spouse,Transport-moving,Wife,White,Female,4064,0,22,United-States,<50k


In [3]:
df.dtypes

age                 int64
workclass          object
fnlwgt              int64
education          object
education-num     float64
marital-status     object
occupation         object
relationship       object
race               object
sex                object
capital-gain        int64
capital-loss        int64
hours-per-week      int64
native-country     object
salary             object
dtype: object

In [4]:
dep_var = 'salary'

cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [FillMissing, Categorify]

In [5]:
test = TabularList.from_df(df.iloc[800:1000].copy(), path=path, cat_names=cat_names, cont_names=cont_names)

In [6]:
test[0]

age                               45
workclass                    Private
fnlwgt                         96975
education               Some-college
education-num                    NaN
marital-status              Divorced
occupation         Handlers-cleaners
relationship               Unmarried
race                           White
sex                           Female
capital-gain                       0
capital-loss                       0
hours-per-week                    40
native-country         United-States
salary                          <50k
Name: 800, dtype: object

In [8]:
data = (TabularList.from_df(df, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(800,1000)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

In [9]:
data.show_batch(rows=10)

workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,target
Private,Assoc-acdm,Married-civ-spouse,Tech-support,Husband,White,False,54.0000,96062.0000,12.0000,>=50k
Private,Some-college,Married-civ-spouse,Adm-clerical,Wife,White,False,52.0000,389270.0000,10.0000,>=50k
Self-emp-not-inc,Some-college,Divorced,Other-service,Unmarried,White,False,30.0000,123397.0000,10.0000,<50k
Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,False,26.0000,133766.0000,9.0000,<50k
Private,10th,Married-civ-spouse,Transport-moving,Husband,White,False,54.0000,175912.0000,6.0000,<50k
Self-emp-not-inc,HS-grad,Married-civ-spouse,Sales,Wife,White,False,38.0000,423616.0000,9.0000,>=50k
Private,HS-grad,Never-married,Sales,Not-in-family,White,False,21.0000,184543.0000,9.0000,<50k
Self-emp-not-inc,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,False,28.0000,183523.0000,9.0000,<50k
?,HS-grad,Separated,?,Not-in-family,Asian-Pac-Islander,False,45.0000,112453.0000,9.0000,<50k
Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,False,59.0000,158813.0000,10.0000,>=50k


In [10]:
learn = tabular_learner(data, emb_drop=0.04, layers=[5, 5, 5], metrics=accuracy)

In [11]:
learn.fit(5, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.365393,0.380859,0.795000,00:03
1,0.357036,0.360171,0.835000,00:03
2,0.357141,0.376896,0.825000,00:03
3,0.365062,0.371819,0.830000,00:03
4,0.350117,0.358102,0.840000,00:03


In [12]:
learn.model

TabularModel(
  (embeds): ModuleList(
    (0): Embedding(10, 6)
    (1): Embedding(17, 8)
    (2): Embedding(8, 5)
    (3): Embedding(16, 8)
    (4): Embedding(7, 5)
    (5): Embedding(6, 4)
    (6): Embedding(3, 3)
  )
  (emb_drop): Dropout(p=0.04, inplace=False)
  (bn_cont): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): Linear(in_features=42, out_features=5, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=5, out_features=5, bias=True)
    (4): ReLU(inplace=True)
    (5): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=5, out_features=5, bias=True)
    (7): ReLU(inplace=True)
    (8): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Linear(in_features=5, out_features=2, bias=True)
  )
)

In [14]:
learn.lr_find()

epoch,train_loss,valid_loss,accuracy,time


LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


In [16]:
learn.fit_one_cycle(10)

epoch,train_loss,valid_loss,accuracy,time
0,0.363758,0.358133,0.845000,00:03
1,0.347675,0.355949,0.845000,00:03
2,0.350580,0.353956,0.840000,00:03
3,0.356411,0.355117,0.835000,00:03
4,0.361173,0.351593,0.835000,00:03
5,0.350823,0.352922,0.830000,00:03
6,0.342527,0.351947,0.835000,00:03
7,0.355571,0.354489,0.830000,00:03
8,0.339016,0.353507,0.825000,00:03
9,0.341408,0.353142,0.820000,00:03


In [17]:
learn.fit(100)

epoch,train_loss,valid_loss,accuracy,time
0,0.358597,0.347672,0.830000,00:03
1,0.360179,0.350817,0.835000,00:03
2,0.352381,0.350452,0.830000,00:03
3,0.346319,0.355258,0.820000,00:03
4,0.362083,0.350890,0.825000,00:03
5,0.356874,0.352983,0.825000,00:03
6,0.348349,0.351253,0.845000,00:03
7,0.358209,0.353719,0.830000,00:03
8,0.345189,0.347160,0.845000,00:03
9,0.350973,0.348359,0.840000,00:03


In [42]:
learn.show_results()

workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,target,prediction
Private,Some-college,Divorced,Handlers-cleaners,Unmarried,White,True,45.0000,96975.0000,10.0000,<50k,<50k
Self-emp-inc,Prof-school,Married-civ-spouse,Prof-specialty,Husband,White,True,46.0000,192779.0000,10.0000,>=50k,>=50k
Private,Assoc-voc,Divorced,#na#,Not-in-family,White,True,36.0000,376455.0000,10.0000,<50k,<50k
Federal-gov,Bachelors,Never-married,Tech-support,Not-in-family,White,True,25.0000,50053.0000,10.0000,<50k,<50k
Private,Bachelors,Married-civ-spouse,#na#,Husband,White,True,37.0000,164526.0000,10.0000,<50k,>=50k


In [38]:
clas, probs

(tensor(0), tensor([0.5279, 0.4721]))